In [28]:
#import the dataset 
from keras.datasets import fashion_mnist

In [29]:
#import the required libraries 
from matplotlib import pyplot
import numpy as np

In [30]:
#load the data into train and test 
(X_train,y_train),(X_test,y_test)=fashion_mnist.load_data()
#pyplot.imshow(X_train[2])

In [31]:
noOfImages = X_train.shape[0]
X_train = (1.0/255)*np.array([X_train[i].flatten() for i in range(0,noOfImages)])
X_train = np.array([X_train[i].flatten() for i in range(0,noOfImages)])
noOftestImages= X_test.shape[0]
X_test =(1.0/255)*np.array([X_test[i].flatten() for i in range(0,noOftestImages)])
X_test =np.array([X_test[i].flatten() for i in range(0,noOftestImages)])
print(X_test.shape)


(10000, 784)


In [6]:
print(X_train[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.00392157 0.         0.         0.05098039 0.28627451 

In [32]:
def softmax(X):
  X=np.exp(X)
  sum=np.sum(X,axis=0)
  return X/sum 


In [33]:
def sigmoidFunc(X):

  return 1.0/(1.0+np.exp(-X))
  #return res

In [34]:
def gDash(X):
  return sigmoidFunc(X)*(1-sigmoidFunc(X))

In [35]:
def forwardPropagation(X,parameters):
  preactivation={}
  activation={}
  activation['h0']=X.T
  #print(activation['h0'].shape)
  for k in range(1,noOfHiddenLayers+1):
    preactivation['a'+str(k)]=np.dot(parameters['W'+str(k)],activation['h'+str(k-1)])+parameters['b'+str(k)]
    activation['h'+str(k)]=sigmoidFunc(preactivation['a'+str(k)])
    #print('h size '+str(k),activation['h'+str(k)].shape)
    #print('a'+str(k),preactivation['a'+str(k)])
  preactivation['a'+str(noOfHiddenLayers+1)]=np.dot(parameters['W'+str(noOfHiddenLayers+1)],activation['h'+str(noOfHiddenLayers)])+parameters['b'+str(noOfHiddenLayers+1)]
  y=softmax(preactivation['a'+str(noOfHiddenLayers+1)])    
  #print("a last" ,preactivation['a'+str(noOfHiddenLayers+1)])
  return (preactivation,activation,y)



In [36]:
def backPropagation(parameters,activation,preactivation,yhat,X,y_train):
  grads={}
  eIndicator=np.zeros((10,X.shape[0]))
  eIndicator[y_train,np.arange(X.shape[0])]=1
  #print(y_train)
  #print(eIndicator)
  #print(eIndicator.shape)
  grads['a'+str(noOfHiddenLayers+1)]= -(eIndicator-yhat)
  #print(grads['a'+str(noOfHiddenLayers+1)].shape)
  for j in range(noOfHiddenLayers+1,0,-1):
    grads['W'+str(j)]= np.dot(grads['a'+str(j)],activation['h'+str(j-1)].T)
    #grads['W'+str(j)]= np.dot(activation['h'+str(j-1)],grads['a'+str(j)].T).T
    #print(grads['W'+str(j)].shape)
    grads['b'+str(j)]= np.sum(grads['a'+str(j)],axis=1,keepdims=True)
    ###grads['h'+str(j-1)]=np.dot(grads['W'+str(j)].T,grads['a'+str(j)]) #error
    grads['h'+str(j-1)]=np.dot(parameters['W'+str(j)].T,grads['a'+str(j)])
    #print('h'+str(j-1),grads['h'+str(j-1)].shape)
    #print('a'+str(j-1),preactivation['a'+str(j-1)].shape)
    if (j!=1):
      grads['a'+str(j-1)]=grads['h'+str(j-1)]*gDash(preactivation['a'+str(j-1)])
  return grads
#backPropagation()

In [37]:
def Loss(yhat,y_train,X):
  eIndicator=np.zeros((10,X.shape[0]))
  eIndicator[y_train,np.arange(X.shape[0])]=1
  eIndicator=eIndicator*yhat
  eIndicator=eIndicator.sum(axis=0)
  eIndicator=np.log(eIndicator)
  return -sum(eIndicator)


In [51]:
noOfHiddenLayers=2
l=10 #output classes
noOfneuronsEach=[120,20]
inputNeuronSize=X_train.shape[1]
print(inputNeuronSize)
parameters={}
eta=1
#eta=0.1
#input W
def gradDescent():
  #batchSize=1000
  iterations=300
  #initialization
  parameters['W'+str(1)] =np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[0], inputNeuronSize))
  parameters['b'+str(1)]= np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[0],1))
  #parameters['W'+str(1)] =0+1.5*np.random.randn(noOfneuronsEach[0], inputNeuronSize)
  #parameters['b'+str(1)]= 0+1.5*np.random.randn(noOfneuronsEach[0],1)
  for i in range(2,noOfHiddenLayers+1):
    parameters['W'+str(i)] = np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[i-1], noOfneuronsEach[i-2]))
    parameters['b'+str(i)]= np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[i-1],1))
    #parameters['W'+str(i)] = 0+1.5*np.random.randn(noOfneuronsEach[i-1], noOfneuronsEach[i-2])
    #parameters['b'+str(i)]= 0+1.5*np.random.randn(noOfneuronsEach[i-1],1)
  #Output W
  parameters['W'+str(noOfHiddenLayers+1)] = np.random.uniform(low=-0.5,high=0.5,size=(l, noOfneuronsEach[-1]))
  parameters['b'+str(noOfHiddenLayers+1)]= np.random.uniform(low=-0.5,high=0.5,size=(l,1))
  #parameters['W'+str(noOfHiddenLayers+1)] = 0+1.5*np.random.randn(l, noOfneuronsEach[-1])
  #parameters['b'+str(noOfHiddenLayers+1)]= 0+1.5*np.random.randn(l,1)
  t=0
  while(t < iterations):
    print("iter ",t)
    #print(parameters)
    #mini=0
    #while(mini<(noOfImages/batchSize)):
      #print("Epoch",mini)
      #X_mini=X_train[(mini*batchSize):((mini+1)*batchSize-1)]
      #y_mini=y_train[(mini*batchSize):((mini+1)*batchSize-1)]
    preactivation,activation,yhat=forwardPropagation(X_train,parameters)
      #print(preactivation)
      #print(activation)
    #print("y_pred",yhat)
    print("loss",Loss(yhat,y_train,X_train))
    gradients=backPropagation(parameters,activation,preactivation,yhat,X_train,y_train)
    #print("gradients",gradients)
    for i in range(1,noOfHiddenLayers+2):
      parameters['W'+str(i)] -=  eta*(1.0/X_train.shape[0])*gradients['W'+str(i)]#parameters['W'+str(i)] -=  eta*(1.0/X_train.shape[0])*gradients['W'+str(i)]
      parameters['b'+str(i)] -= eta*(1.0/X_train.shape[0])*gradients['b'+str(i)]#parameters['b'+str(i)] -= eta*(1.0/X_train.shape[0])*gradients['b'+str(i)]
    #mini+=1
    #_,_,yhat=forwardPropagation(X_train,parameters)
    
    #print(yhat)
    t+=1
gradDescent()
print(parameters)
#print(gradients)



784
iter  0
loss -154997.02400733135
iter  1
loss -137447.60304714643
iter  2
loss -132578.50028885275
iter  3
loss -129888.84761075344
iter  4
loss -127316.21533296791
iter  5
loss -124616.48549251058
iter  6
loss -121751.40315815431
iter  7
loss -118723.3530597767
iter  8
loss -115579.41768477685
iter  9
loss -112403.75758766153
iter  10
loss -109279.60108687232
iter  11
loss -106255.88530774502
iter  12
loss -103351.48096244877
iter  13
loss -100573.8746888065
iter  14
loss -97926.58170678712
iter  15
loss -95409.56434573549
iter  16
loss -93019.10343926252
iter  17
loss -90748.53876088017
iter  18
loss -88589.73089854687
iter  19
loss -86534.6106756568
iter  20
loss -84576.15167333823
iter  21
loss -82708.59830842337
iter  22
loss -80927.25922864968
iter  23
loss -79228.23786627903
iter  24
loss -77608.23225852141
iter  25
loss -76064.33253845594
iter  26
loss -74593.75658353124
iter  27
loss -73193.59163084727
iter  28
loss -71860.65922852671
iter  29
loss -70591.54279930856
iter 

In [52]:
def predict(X_test,y_test):
  _,_,y_hat=forwardPropagation(X_test,parameters)
  y_hat=y_hat.argmax(axis=0)
  correctPred=np.sum(y_hat==y_test)
  print("Accuracy is ", correctPred/X_test.shape[0]*100,"%")
  return (y_hat,correctPred)
print(predict(X_train,y_train))
print(predict(X_test,y_test))

Accuracy is  82.0 %
(array([9, 0, 3, ..., 3, 0, 5]), 49200)
Accuracy is  80.58999999999999 %
(array([9, 2, 1, ..., 8, 1, 5]), 8059)
